In [1]:
from odps import ODPS
from odps import options
import oss2
from odps.df import DataFrame
import datetime
import pandas as pd
import numpy as np
import threading
import queue
import time

# connect信息
from ACCESS import ACCESS_ID
from ACCESS import SECRET_ACCESS_KEY
from ACCESS import ODPS_PROJECT
from ACCESS import ODPS_ENDPOINT

# # 此为外网地址
OSS_ENDPOINT = 'http://oss-cn-shanghai.aliyuncs.com'
OSS_BUCKET = 'hypertrons'

# oss实例
auth = oss2.Auth(ACCESS_ID, SECRET_ACCESS_KEY)
bucket = oss2.Bucket(auth, OSS_ENDPOINT, OSS_BUCKET)

# 创建odps实例
o = ODPS(ACCESS_ID, SECRET_ACCESS_KEY, project=ODPS_PROJECT, endpoint=ODPS_ENDPOINT)
options.tunnel.limit_instance_tunnel = False
# options.read_timeout = 3600000

hints = {'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'}

In [44]:
drop_sql = '''
DROP TABLE IF EXISTS organizations;
CREATE TABLE IF NOT EXISTS organizations
(
    org_id STRING,
    org_login STRING,
    degree BIGINT
);
'''
o.execute_sql(drop_sql, hints=hints)

select_sql = '''
    set odps.sql.validate.orderby.limit=false;
    INSERT INTO organizations
    SELECT org_id, org_login, degree
    FROM Yue_org_degree
    ORDER BY degree DESC
    LIMIT 100
    '''
o.execute_sql(select_sql, hints=hints)
tmp = DataFrame(o.get_table('organizations'))
tmp.count()

100

In [45]:
select_sql = '''
    set odps.sql.validate.orderby.limit=false;
    SELECT org_id, org_login, degree
    FROM organizations
    ORDER BY degree DESC
    '''
table = o.execute_sql(select_sql, hints=hints)
i=0
with table.open_reader(tunnel=True) as reader:
    for record in reader:
        print(record.org_id, record.org_login, record.degree)
        i += 1
        if i > 10:
            break

1342004 google 2910
131524 mozilla 1953
6154722 microsoft 1831
107424 jenkinsci 1332
6154722 Microsoft 1226
2810941 GoogleCloudPlatform 1219
1459110 IBM 1210
9919 github 1177
25064361 alchemycodelab 1113
66423638 IIM-Creative-Technology 1107
6844498 Azure 1087


In [48]:
drop_sql = '''
DROP TABLE IF EXISTS Yue_PR_issue;
CREATE TABLE IF NOT EXISTS Yue_PR_issue
(
    org_id STRING,
    org_login STRING,
    issue_title STRING,
    issue_body STRING
);
'''
o.execute_sql(drop_sql, hints=hints)

select_sql = '''
    INSERT INTO Yue_PR_issue
    SELECT org_id, org_login, issue_title, issue_body
    FROM ods_github_log
    WHERE type='PullRequestEvent' AND org_id IN (SELECT org_id FROM organizations) AND issue_body IS NOT NULL
'''
o.execute_sql(select_sql, hints=hints)
tmp = DataFrame(o.get_table('Yue_PR_issue'))
tmp.count()

3134810

In [49]:
select_sql = '''
    set odps.sql.validate.orderby.limit=false;
    SELECT org_id, org_login, count(issue_title) AS count
    FROM Yue_PR_issue
    GROUP BY org_id, org_login
    ORDER BY count DESC
    '''
table = o.execute_sql(select_sql, hints=hints)
i=0
with table.open_reader(tunnel=True) as reader:
    for record in reader:
        print(record.org_id, record.org_login, record.count)
        i += 1
        if i > 10:
            break

6154722 microsoft 598737
6844498 Azure 597489
1342004 google 395130
131524 mozilla 387036
6154722 Microsoft 322525
2810941 GoogleCloudPlatform 248023
107424 jenkinsci 148271
1459110 IBM 112646
9919 github 94634
9950313 nodejs 82417
3636186 makersacademy 31757


In [57]:
select_sql = '''
    set odps.sql.validate.orderby.limit=false;
    SELECT distinct org_id, org_login, issue_title, issue_body
    FROM Yue_PR_issue
    '''
i = 0
table = o.execute_sql(select_sql, hints=hints)
with table.open_reader(tunnel=True) as reader:
    for record in reader:
        print(record.org_id, record.org_login, record.issue_title)
        i += 1
        if i > 100:
            break

1004228 Strategery-Inc  Add cache options for prevent underscore in URLs
1004228 Strategery-Inc - Added  validation.
1004228 Strategery-Inc - Color swatches fix.
1004228 Strategery-Inc - Fix for configurable swatches.
1004228 Strategery-Inc - Fix for image.
1004228 Strategery-Inc - Fix for issue #96 in GitHub.
1004228 Strategery-Inc - Fixed default mode.
1004228 Strategery-Inc - Fixed version.
1004228 Strategery-Inc - Last version.
1004228 Strategery-Inc - New version.
1004228 Strategery-Inc - README.md
1004228 Strategery-Inc - RWD $j  validation and disabled jQuery include by default.
1004228 Strategery-Inc - Replaced tabs by spaces, cut large code lines, replaced date function.
1004228 Strategery-Inc - Trigger window > delayed-resize event to fix items height.
1004228 Strategery-Inc - Updated README.
1004228 Strategery-Inc - Updated jQuery IAS - Fixed previous button selector. - Added previo…
1004228 Strategery-Inc - V. 3.2.3
1004228 Strategery-Inc - V.0.2.1 - Warnings fixes.
1004228